# Same as Exp6 but with MaskedRoBERTa model

In [1]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from collections import defaultdict
from scipy import spatial
from IPython.display import HTML, display
import tabulate
import json
import pickle
import re
import nltk

# Dataset Prep

### 1. Test set 

In [2]:
df = pd.read_excel("InputTestSet-Reviews48_Ann.xlsx")

In [3]:
df.head()

,UID,PID,Dec,Sent,MComp,Cat,SubCat
0,0,2019_SJf_XhCqKm,Reject,The authors propose to use k-DPP to select a s...,0,NaN,NaN
1,1,2019_SJf_XhCqKm,Reject,"This paper covers the related work nicely, wit...",0,NaN,NaN
2,2,2019_SJf_XhCqKm,Reject,The rest of the paper are also clearly written.,0,NaN,NaN
3,3,2019_SJf_XhCqKm,Reject,"However, I have some concerns about the propos...",0,NaN,NaN
4,4,2019_SJf_XhCqKm,Reject,"- It is not clear how to define the kernel, th...",0,NaN,NaN


In [4]:
df.shape

(1505, 7)

In [5]:
gt_dict = {}

for i in range(0, df.shape[0]):
    pid = df.loc[i]["PID"]
    if not pid in gt_dict:
        gt_dict[pid] = {"dec": df.loc[i]["Dec"], "mcomp": set(), "not_mcomp": set()}
    if df.loc[i]["MComp"] == 1:
        gt_dict[pid]["mcomp"].add(df.loc[i]["UID"])
    else:
        gt_dict[pid]["not_mcomp"].add(df.loc[i]["UID"])

In [6]:
stats_dict = {"Accept": [0, 0], "Reject": [0, 0]}

for k, v in gt_dict.items():
    #print(len(v["mcomp"]), len(v["not_mcomp"]), v["dec"])
    stats_dict[v["dec"]][0] += len(v["mcomp"])
    stats_dict[v["dec"]][1] += len(v["not_mcomp"])
    
print(stats_dict)

{'Accept': [48, 644], 'Reject': [69, 744]}


In [7]:
test_set = list(gt_dict.keys())
print("TestSet length: %d\n"%len(test_set), test_set)

TestSet length: 32
 ['2019_SJf_XhCqKm', '2017_Bk0MRI5lg', '2020_SyevYxHtDB', '2018_rJBiunlAW', '2020_rkltE0VKwH', '2018_Hki-ZlbA-', '2019_BJx0sjC5FX', '2020_r1e_FpNFDr', '2020_B1lsXREYvr', '2018_SkZxCk-0Z', '2019_rJzoujRct7', '2018_HkfXMz-Ab', '2017_BJ9fZNqle', '2019_SyxZJn05YX', '2017_B1ckMDqlg', '2017_HJ0NvFzxl', '2017_S1_pAu9xl', '2018_SyYYPdg0-', '2017_BJAA4wKxg', '2019_HyVxPsC9tm', '2019_HylTBhA5tQ', '2019_B1l08oAct7', '2018_H135uzZ0-', '2017_H1oyRlYgg', '2017_r1y1aawlg', '2020_r1eX1yrKwB', '2020_Byg79h4tvB', '2019_H1lFZnR5YX', '2020_BkeWw6VFwr', '2018_HyHmGyZCZ', '2018_HyUNwulC-', '2020_HkgsPhNYPS']


In [8]:
for k in test_set:
    print('{:20}{}'.format(k, gt_dict[k]["mcomp"]))

2019_SJf_XhCqKm     {39, 17, 20, 27, 28, 30}
2017_Bk0MRI5lg      {48, 57}
2020_SyevYxHtDB     {76, 87}
2018_rJBiunlAW      {108, 110, 112, 113, 124, 126}
2020_rkltE0VKwH     {160, 155, 184, 159}
2018_Hki-ZlbA-      {267, 235, 236, 271}
2019_BJx0sjC5FX     {292, 287}
2020_r1e_FpNFDr     {312, 322, 315, 308}
2020_B1lsXREYvr     {376, 401}
2018_SkZxCk-0Z      {449, 443, 445, 486}
2019_rJzoujRct7     {518, 519}
2018_HkfXMz-Ab      {573, 566}
2017_BJ9fZNqle      {627, 623, 615}
2019_SyxZJn05YX     {672, 673, 657, 669, 671}
2017_B1ckMDqlg      {714, 707}
2017_HJ0NvFzxl      {739}
2017_S1_pAu9xl      {792, 809, 810, 806}
2018_SyYYPdg0-      {834, 867, 868, 869, 870, 872, 873, 844, 830}
2017_BJAA4wKxg      {884}
2019_HyVxPsC9tm     {931, 933, 905, 909, 912, 913, 919, 926}
2019_HylTBhA5tQ     {972, 950}
2019_B1l08oAct7     {994, 996, 1064, 1004, 1007, 1044, 1047, 1048, 1055}
2018_H135uzZ0-      {1072, 1079}
2017_H1oyRlYgg      set()
2017_r1y1aawlg      {1125, 1162, 1100, 1102, 1168}
2020_r1eX1y

In [9]:
sents_for_test = defaultdict(list)

for i in range(0, df.shape[0]):
    pid = df.loc[i]["PID"]
    test_sent_raw = str(df.loc[i]["Sent"])
    
    # Replace URLs with [URL]
    test_sent_raw = re.sub(r'http[s]?://[a-zA-z\.\-/0-9~]*', "[URL]", test_sent_raw)
    test_sent_raw = re.sub(r'papers.nips.cc/paper/[a-zA-z\.\-/0-9~]*', "[URL]", test_sent_raw)
    test_sent_raw = re.sub(r'arxiv.org/[a-zA-z\.\-/0-9~]*', "[URL]", test_sent_raw)
    
    sents_for_test[pid].append((df.loc[i]["UID"], test_sent_raw))

### 2. Train Set

In [10]:
df_train = pd.read_excel("InputTrainSet-Reviews7_Ann.xlsx")
df_train.head()

,UID,PID,Dec,Sent,MComp,Cat,SubCat
0,243,2020_ryen_CEFwr,Reject,It extends this approach by introducing an add...,0,None,None
1,179,2018_H1LAqMbRW,Reject,"Experimentally, the results are rather weak co...",0,None,None
2,157,2017_HyTqHL5xg,Accept,The experiments are interesting but I'm still ...,0,None,None
3,146,2017_HyTqHL5xg,Accept,Section 2.2 says they do the latter in the int...,0,None,None
4,90,2017_ByToKu9ll,Reject,4)This paper proposed an improved version of t...,0,None,None


In [12]:
df_train.shape

(296, 7)

In [15]:
df_train[df_train["MComp"] == 1].shape

(26, 7)

In [16]:
train_sets = {"mcomp": [], "non_mcomp": []}

In [17]:
for i in range(0, df_train.shape[0]):
    pid = df_train.loc[i]["PID"]
    train_sent_raw = str(df_train.loc[i]["Sent"])
    
    type_comp = df_train.loc[i]["MComp"]
    
    if type_comp == 1:
        train_sets["mcomp"].append(train_sent_raw)
    else:
        train_sets["non_mcomp"].append(train_sent_raw)

In [18]:
len(train_sets["mcomp"]), len(train_sets["non_mcomp"])

(26, 270)

## Load entities

In [19]:
with open("entities_dict_smaller", "r") as f:
    entity_dict = json.load(f)

In [20]:
set(entity_dict.values())

{'Material', 'Method', 'Metric', 'Task'}

In [21]:
list(entity_dict.items())[0:20]

[('convolutional neural networks', 'Method'),
 ('convnets', 'Method'),
 ('recognition', 'Task'),
 ('visual recognition tasks', 'Task'),
 ('age estimation', 'Task'),
 ('head pose estimation', 'Task'),
 ('multi - label classification', 'Task'),
 ('semantic segmentation', 'Task'),
 ('classification', 'Task'),
 ('deep convnets', 'Method'),
 ('dldl', 'Method'),
 ('feature learning', 'Task'),
 ('deep learning', 'Method'),
 ('image classification', 'Task'),
 ('deep learning methods', 'Method'),
 ('image classification tasks', 'Task'),
 ('human pose estimation', 'Task'),
 ('convnet', 'Method'),
 ('recognition tasks', 'Task'),
 ('ensemble', 'Method')]

In [22]:
entity_key_map = {}
for i in entity_dict:
    s = re.sub('[^0-9a-zA-Z,:;.?!\- ]+', '', i)
    while s.find("  ") > -1:
        s = s.replace("  ", " ")
    if len(s) > 2:
        cl = re.sub('[^0-9a-zA-Z ]+', '', i)
        while cl.find("  ") > -1:
            cl = cl.replace("  ", " ")
        entity_key_map[cl.strip()] = i
print(len(entity_key_map))

1784


In [23]:
coun = 0
for i in entity_dict:
    if len(i) < 5:
        coun +=1
#         print(i)
print(coun)

212


In [24]:
list(entity_key_map.items())[0:5]

[('convolutional neural networks', 'convolutional neural networks'),
 ('convnets', 'convnets'),
 ('recognition', 'recognition'),
 ('visual recognition tasks', 'visual recognition tasks'),
 ('age estimation', 'age estimation')]

In [25]:
from collections import Counter
c = Counter(entity_dict.values())
c

Counter({'Method': 1191, 'Task': 289, 'Metric': 158, 'Material': 165})

In [26]:
# dir(c)
reverse_map = defaultdict(list)

for k, v in entity_dict.items():
    reverse_map[v].append(k)

In [27]:
# for i in reverse_map["Task"]:
#     print(i)

In [28]:
"MNIST" in entity_key_map, "mnist" in entity_key_map

(False, True)

## A. RoBERTa trained on SciLit

In [29]:
import spacy
import torch

In [30]:
!pip3.7 list | grep -E 'transformers|tokenizers'

spacy-transformers            0.6.2
tokenizers                    0.7.0
transformers                  2.9.0
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.


In [31]:
from transformers import AutoTokenizer, AutoModel

In [32]:
tokenizer = AutoTokenizer.from_pretrained("./trained_lm/MaskedRoBERTa/")
model = AutoModel.from_pretrained("./trained_lm/MaskedRoBERTa/")

In [33]:
import transformers
print(transformers.__version__)

2.9.0


In [34]:
def embed_text_using_roberta(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    return last_hidden_states

In [35]:
def mask_entities(sentence, replace_with_dataset=True):
    cleaned_sent = re.sub('[^0-9a-zA-Z,:;.?!\- ]+', ' ', sentence)
    while cleaned_sent.find("  ") > -1:
        cleaned_sent = cleaned_sent.replace("  ", " ")
    
    entity_key_map_keys = list(entity_key_map.keys()) # As we will be dunamically adding entries to this dict an dthat will throw an error.
    entities_found = []
    for i in entity_key_map_keys:
        if cleaned_sent.find(" " + i + " ") > -1:
            entities_found.append(i)
        elif cleaned_sent.lower().find(" " + i + " ") > -1:
            found_idx = cleaned_sent.lower().find(" " + i + " ")
            entity_dict[cleaned_sent[found_idx:found_idx+len(" " + i + " ")]] = entity_dict[i]
            entity_key_map[cleaned_sent[found_idx:found_idx+len(" " + i + " ")]] = entity_key_map[i]
    
    entities_found.sort(key=lambda s: len(s))
    len_sorted_entities = entities_found.copy()
    
    subset_entities = []
    # Remove subset entities (eg: Among cnn and 3-layer-cnn, prefer the latter)
    for fe in len_sorted_entities:
        for other_ent in len_sorted_entities:
            if fe != other_ent and other_ent.find(fe) > -1:
                subset_entities.append(fe)
                break
    for se in subset_entities:
        len_sorted_entities.remove(se)
    for maxents in len_sorted_entities:
        mask_name = " " + entity_dict[entity_key_map[i]].lower() + " "
        if replace_with_dataset:
            if mask_name == " material ":
                mask_name = " dataset "
        cleaned_sent = cleaned_sent.replace(" " + maxents + " ", mask_name)
    words_cleaned = nltk.word_tokenize(cleaned_sent)
    dups_removed = [v for i, v in enumerate(words_cleaned) if i == 0 or v != words_cleaned[i-1]]
    new_dup_removed_sent = " ".join(dups_removed)
    return new_dup_removed_sent.strip()

#     #print(cleaned_sent)
#     for i in entity_key_map:
#         if cleaned_sent.find(" " + i + " ") > -1:
#             #print("Substituting ent: {} with mask: {}".format(i, entity_dict[entity_key_map[i]].lower()))
#             cleaned_sent = cleaned_sent.replace(i, entity_dict[entity_key_map[i]].lower())
#     return cleaned_sent

In [36]:
nlp = spacy.load('en_core_web_sm')
sp_toks = ["result", "method", "task", "dataset", "metric", "baseline", "fair", "unfair"]

In [37]:
def extract_chunks_using_spacy_dp(conssentence, replace_with_dataset=True):
    
    conssentence = mask_entities(conssentence, replace_with_dataset)
#     print(conssentence)
    doc = nlp(conssentence)
    verb_subtree = []

    for s in doc.sents:
#         find_special_tokens = {"compar": [], "result": [], "method": [], "technique": [], "task": [], "dataset": [], "material": [], "metric": []}
        find_special_tokens = {"compar": [], "result": [], "method": [], "baseline": [], "task": [], 
                               "dataset": [],  "metric": [], "unfair": [], "fair": []}

        for tok in s:

            if tok.text.lower().startswith("compar"):
                find_special_tokens["compar"].append(tok)
            else:
                for k in sp_toks:
                    if tok.text.lower().startswith(k):
                        find_special_tokens[k].append(tok)
                        break

        verb_tokens = []
        if find_special_tokens["compar"]:
            for t in find_special_tokens["compar"]:
#                     verb_subtree.append(t.subtree)
                if t == s.root:
                    simplified_sent = ""
                    for chh in t.lefts:
                        simplified_sent = simplified_sent + " " + chh.text
                    simplified_sent = simplified_sent + " " + t.text
                    for chh in t.rights:
                        simplified_sent = simplified_sent + " " + chh.text
#                         print("SIMP: ", simplified_sent)
                    verb_subtree.append(simplified_sent)
                else:
                    verb_subtree.append(t.subtree)
        else:
            for k in sp_toks:
                for i in find_special_tokens[k]:
                    local_vt = []
                    for j in i.ancestors:
                        if j.pos_ == "NOUN":
                            local_vt.append(j)
                    if not local_vt:
                        for j in i.ancestors:
                            if j.pos_ == "VERB":
                                local_vt.append(j)
                    verb_tokens = verb_tokens + local_vt


            for i in verb_tokens:
                verb_subtree.append(i.subtree)

    eecc = []
    for i in verb_subtree:
        if type(i) == str:
            eecc.append(i)
        else:
            local_chunk = ""
            for lcaltok in i:
                local_chunk = local_chunk + " " + lcaltok.text
            eecc.append(local_chunk)
#     if not eecc:
#         print(conssentence)
    return list(set(eecc))


### Computing vectors of the initial training pool of sentences

In [49]:
train_pool_roberta_vecs = {"mcomp": [], "non_mcomp": []}
single_train_pool_roberta_vecs = {"mcomp": [], "non_mcomp": []}

In [ ]:
for i in train_sets["mcomp"]:
    mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(i)
    if mcomp_chunks_from_sent:
        final_chunks = mcomp_chunks_from_sent
    else:
        final_chunks = [i]
    
    for single_chunk in final_chunks:
        vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
        train_pool_roberta_vecs["mcomp"].append(vec/norm(vec))
    
    collated_chunk = " ".join(final_chunks)
    vec = embed_text_using_roberta(collated_chunk.strip()).mean(1).detach().numpy()
    single_train_pool_roberta_vecs["mcomp"].append(vec/norm(vec))


for i in train_sets["non_mcomp"]:
    mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(i)
    if mcomp_chunks_from_sent:
        final_chunks = mcomp_chunks_from_sent
    else:
        final_chunks = [i]
    
    for single_chunk in final_chunks:
        vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
        train_pool_roberta_vecs["non_mcomp"].append(vec/norm(vec))
    
    collated_chunk = " ".join(final_chunks)
    vec = embed_text_using_roberta(collated_chunk.strip()).mean(1).detach().numpy()
    single_train_pool_roberta_vecs["non_mcomp"].append(vec/norm(vec))


In [ ]:
roberta_vectors = defaultdict(dict)
skip_uids = []

for pid in gt_dict:
    roberta_vectors[pid] = {}
    
    for mcs in gt_dict[pid]["mcomp"]:
        try:
            mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(df.loc[mcs]["Sent"])
            if mcomp_chunks_from_sent:
                final_chunks = mcomp_chunks_from_sent
            else:
                final_chunks = [df.loc[mcs]["Sent"]]
            
            roberta_vectors[pid][mcs] = []
            for single_chunk in final_chunks:
                vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
                roberta_vectors[pid][mcs].append(vec / norm(vec))
        except Exception as ex:
            print(pid, mcs, df.loc[mcs]["Sent"])
            skip_uids.append(mcs)
    
    for mcs in gt_dict[pid]["not_mcomp"]:
        try:
            mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(df.loc[mcs]["Sent"])
            if mcomp_chunks_from_sent:
                final_chunks = mcomp_chunks_from_sent
            else:
                final_chunks = [df.loc[mcs]["Sent"]]
            
            roberta_vectors[pid][mcs] = []
            for single_chunk in final_chunks:
                vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
                roberta_vectors[pid][mcs].append(vec / norm(vec))
        except Exception as ex:
            print(pid, mcs, df.loc[mcs]["Sent"])
            skip_uids.append(mcs)

In [ ]:
mcomp_sentences = {}
not_mcomp_sentences = {}

for pid in gt_dict:
    for mcs in gt_dict[pid]["mcomp"]:
        if not mcs in skip_uids:
            mcomp_sentences[mcs] = pid
    for mcs in gt_dict[pid]["not_mcomp"]:
        if not mcs in skip_uids:
            not_mcomp_sentences[mcs] = pid
print(len(mcomp_sentences), len(not_mcomp_sentences))

In [ ]:
mean_sim_with_mcomp = defaultdict(dict)
mean_sim_with_not_mcomp = defaultdict(dict)
max_sim_with_mcomp = defaultdict(dict)
max_sim_with_not_mcomp = defaultdict(dict)

mean_at_k = ["1", "3", "5", "7", "10", "26"]


for sid in mcomp_sentences:
    
    # 1. With other training set mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["mcomp"]:
        for cvec2 in roberta_vectors[mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])
    
    
    # 2. With other training set non_mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["non_mcomp"]:
        for cvec2 in roberta_vectors[mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_not_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_not_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_not_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_not_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])




for sid in not_mcomp_sentences:
    
    # 1. With other training set mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["mcomp"]:
        for cvec2 in roberta_vectors[not_mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])
    
    
    # 2. With other training set non_mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["non_mcomp"]:
        for cvec2 in roberta_vectors[not_mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_not_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_not_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_not_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_not_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])


In [43]:
def precision_at_k(sent_sim_dict, k=10, sim_type="max", mean_at=1):
    """mean_at can take values: 1, 3, 5, 7, 10, 26"""
    
    if sim_type == "max":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["max"], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        fp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
            else:
                fp += 1
        return tp/(tp+fp)
            
    elif sim_type == "mean":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["mean_"+str(mean_at)], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        fp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
            else:
                fp += 1
        return tp/(tp+fp)

In [44]:
def recall_at_k(sent_sim_dict, k=10, sim_type="max", mean_at=1):
    if sim_type == "max":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["max"], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
        return tp/min(k, 117)
            
    elif sim_type == "mean":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["mean_"+str(mean_at)], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
        return tp/min(k, 117)

### Based on max similarity

In [41]:
print("P and R based on max similarity with initial pool of \nmeaningful sentences:")

patk = [1, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600]
res_table = [["Precision at:"] + patk, ["Val"]]

for k in patk:
    v1 = precision_at_k(max_sim_with_mcomp, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1, 2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))



ratk = [117, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1388]
res_table = [["Recall at:"] + ratk, ["Val"]]

for k in ratk:
    v1 = recall_at_k(max_sim_with_mcomp, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1,2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))

P and R based on max similarity with initial pool of 
meaningful sentences:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.9,0.75,0.48,0.35,0.34,0.28,0.26,0.21,0.17,0.15,0.14


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.34,0.36,0.44,0.53,0.58,0.63,0.69,0.76,0.82,0.86,0.91,0.97,1


### Based on mean similarity

In [42]:
print("P and R based on avg similarity with initial pool of \nmeaningful sentences:\n\n")


mean_at_j = [1, 5, 10, 26]

for jjj in mean_at_j:
    print("Avg of top {}:".format(jjj))

    patk = [1, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600]
    res_table = [["Precision at:"] + patk, ["Val"]]

    for k in patk:
        v1 = precision_at_k(mean_sim_with_mcomp, k, sim_type="mean", mean_at=jjj) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
        res_table[1].append(round(v1, 2))

    display(HTML(tabulate.tabulate(res_table, tablefmt='html')))



    ratk = [117, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1388]
    res_table = [["Recall at:"] + ratk, ["Val"]]

    for k in ratk:
        v1 = recall_at_k(mean_sim_with_mcomp, k, sim_type="mean", mean_at=jjj) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
        res_table[1].append(round(v1,2))

    display(HTML(tabulate.tabulate(res_table, tablefmt='html')))
    print("\n\n")

P and R based on avg similarity with initial pool of 
meaningful sentences:


Avg of top 1:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.9,0.75,0.48,0.35,0.34,0.28,0.26,0.21,0.17,0.15,0.14


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.34,0.36,0.44,0.53,0.58,0.63,0.69,0.76,0.82,0.86,0.91,0.97,1





Avg of top 5:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.8,0.7,0.5,0.4,0.38,0.33,0.27,0.22,0.18,0.15,0.14


...

Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.4,0.45,0.46,0.39,0.37,0.32,0.28,0.24,0.19,0.16,0.14


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.37,0.41,0.49,0.62,0.65,0.67,0.71,0.75,0.8,0.88,0.94,0.98,1





Avg of top 26:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.1,0.2,0.2,0.25,0.28,0.3,0.28,0.23,0.2,0.17,0.16


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.28,0.38,0.49,0.58,0.69,0.74,0.8,0.84,0.88,0.92,0.95,0.98,1


### DIff of max similarity of meaningful and non-meaningful sentences

In [46]:
diff_max = {}
preserve_non_negative_max = {}

for x in max_sim_with_mcomp:
    diff_max_sim_for_x = max_sim_with_mcomp[x]["max"] - max_sim_with_not_mcomp[x]["max"]
    diff_max[x] = {"max": diff_max_sim_for_x}
    
    if diff_max_sim_for_x > 0:
        preserve_non_negative_max[x] = {"max": max_sim_with_mcomp[x]["max"]}
    else:
        preserve_non_negative_max[x] = {"max": 0}

In [44]:
print("P and R based on diff of max sim with initial pool of \n meaningful & non meaningful sents:")

patk = [1, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600]
res_table = [["Precision at:"] + patk, ["Val"]]

for k in patk:
    v1 = precision_at_k(diff_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1, 2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))



ratk = [117, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1388]
res_table = [["Recall at:"] + ratk, ["Val"]]

for k in ratk:
    v1 = recall_at_k(diff_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1,2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))

P and R based on diff of max sim with initial pool of 
 meaningful & non meaningful sents:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.6,0.6,0.54,0.39,0.37,0.34,0.3,0.25,0.21,0.18,0.16


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.37,0.44,0.51,0.64,0.72,0.75,0.82,0.82,0.85,0.89,0.89,0.93,0.97


In [45]:
print("P and R based on preserving non-neg max sim with initial pool of \n meaningful & non meaningful sents:")

patk = [1, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600]
res_table = [["Precision at:"] + patk, ["Val"]]

for k in patk:
    v1 = precision_at_k(preserve_non_negative_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1, 2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))



ratk = [117, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1388]
res_table = [["Recall at:"] + ratk, ["Val"]]

for k in ratk:
    v1 = recall_at_k(preserve_non_negative_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1,2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))

P and R based on preserving non-neg max sim with initial pool of 
 meaningful & non meaningful sents:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.9,0.7,0.5,0.41,0.37,0.34,0.48,0.39,0.29,0.23,0.2


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.37,0.44,0.83,1,1,1,1,1,1,1,1,1,1


In [48]:
print("P and R based on preserving non-neg max sim with initial pool of \n meaningful & non meaningful sents:")

patk = [1, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600]
res_table = [["Precision at:"] + patk, ["Val"]]

for k in patk:
    v1 = precision_at_k(preserve_non_negative_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1, 2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))



ratk = [117, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1388]
res_table = [["Recall at:"] + ratk, ["Val"]]

for k in ratk:
    v1 = recall_at_k(preserve_non_negative_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1,2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))

P and R based on preserving non-neg max sim with initial pool of 
 meaningful & non meaningful sents:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.9,0.7,0.5,0.41,0.37,0.34,0.48,0.39,0.29,0.23,0.2


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.37,0.44,0.83,1,1,1,1,1,1,1,1,1,1


# ==========================================================